### 两独立样本均值比较：合并方差、SE 与 t 检验（Student vs Welch）

#### 1. 背景与建模（解决什么问题）
- **问题类型**：比较两组“独立”样本的**总体均值是否相同**。  
  典型场景：对照组 vs 处理组、A/B 测试、两种工艺的产品指标等。
- **数据假设**：每组来自正态总体（或样本量足够大近似正态），两组独立。  
- **检验目标**：  
  - 双侧：$H_0:\ \mu_1=\mu_2$；$H_1:\ \mu_1\ne\mu_2$（是否存在差异）。  
  - 单侧：$H_0:\ \mu_1=\mu_2$；$H_1:\ \mu_1>\mu_2$ 或 $H_1:\ \mu_1<\mu_2$（是否更大/更小）。

---

#### 2. 记号与样本统计量
样本：第 1 组 $X_1,\dots,X_{n_1}$，第 2 组 $Y_1,\dots,Y_{n_2}$，独立。  
样本均值与方差：
$$
\bar X,\ S_X^2\ ;\quad \bar Y,\ S_Y^2.
$$
均值差与其“真”标准差（已知方差情况下）：
$$
\Delta=\bar X-\bar Y,\qquad
\mathrm{SD}(\Delta)=\sigma\sqrt{\tfrac{1}{n_1}+\tfrac{1}{n_2}}\quad(\sigma_1^2=\sigma_2^2=\sigma^2).
$$

---

#### 3. 标准误差（SE, Standard Error）
**SE = 估计量的标准差的估计。**  
当总体方差未知，用样本方差来估计 $\mathrm{SD}(\Delta)$：

- **方差齐性（Student t；`equal_var=True`）**  
  先计算**合并方差 (pooled variance)**（按自由度加权）：
  $$
  S_p^2=\frac{(n_1-1)S_X^2+(n_2-1)S_Y^2}{n_1+n_2-2},\qquad E[S_p^2]=\sigma^2.
  $$
  **均值差的 SE**：
  $$
  \mathrm{SE}_{\text{pooled}}=S_p\sqrt{\tfrac{1}{n_1}+\tfrac{1}{n_2}}.
  $$

- **方差不齐（Welch t；`equal_var=False`）**  
  不合并，直接相加：
  $$
  \mathrm{SE}_{\text{Welch}}=\sqrt{\tfrac{S_X^2}{n_1}+\tfrac{S_Y^2}{n_2}}.
  $$

---

#### 4. 合并方差为何成立（直观要点）
- 自由度越大，$S^2$ 越稳定：$\mathrm{Var}(S^2)=\dfrac{2\sigma^4}{n-1}$。  
- 对正态样本：$\dfrac{(n_1-1)S_X^2}{\sigma^2}\sim\chi^2_{n_1-1}$、$\dfrac{(n_2-1)S_Y^2}{\sigma^2}\sim\chi^2_{n_2-1}$ 独立，  
  故其和 $\sim \chi^2_{n_1+n_2-2}$，得到 $E[S_p^2]=\sigma^2$（无偏）。

---

#### 5. t 统计量如何得到 & 为什么服从 t 分布
- **Student t（齐性）**：
  $$
  t=\frac{(\bar X-\bar Y)-(\mu_1-\mu_2)}{\,S_p\sqrt{1/n_1+1/n_2}\,}\ \sim\ t_{\,n_1+n_2-2}\quad(H_0:\mu_1=\mu_2).
  $$
  由“分子为正态、分母含独立的 $\chi^2/\text{df}$ 的平方根”⇒ t 分布（Cochran 定理）。
- **Welch t（不齐性）**：
  $$
  t=\frac{(\bar X-\bar Y)-(\mu_1-\mu_2)}{\sqrt{S_X^2/n_1+S_Y^2/n_2}},
  $$
  自由度用 Welch–Satterthwaite 近似
  $$
  \nu=\frac{\left(\tfrac{S_X^2}{n_1}+\tfrac{S_Y^2}{n_2}\right)^2}{\tfrac{(S_X^2/n_1)^2}{n_1-1}+\tfrac{(S_Y^2/n_2)^2}{n_2-1}},
  $$
  在正态前提下近似服从 $t_\nu$，通常精度很好。

---

#### 6. 假设检验的标准步骤（以双侧为例）
1. **提出假设**：$H_0:\mu_1=\mu_2$；$H_1:\mu_1\ne\mu_2$。  
2. **选择检验**：  
   - 认为方差近似相等 ⇒ Student t（更有力）；  
   - 不确定/明显不等 ⇒ Welch t（更稳健）。  
3. **计算统计量** $t_{\text{obs}}$ 与相应自由度（齐性 $n_1+n_2-2$；不齐性 $\nu$）。  
4. **决策**：  
   - 临界值法：$|t_{\text{obs}}|>t_{\alpha/2}(\text{df})$ ⇒ 拒绝 $H_0$；  
   - $p$ 值法：$p=2\,[1-F_t(|t_{\text{obs}}|;\text{df})]\le\alpha$ ⇒ 拒绝。  
5. **区间估计（等价视角）**：$(\bar X-\bar Y)\pm t_{\alpha/2}(\text{df})\cdot \mathrm{SE}$；区间含 $0$ ⇔ 不拒绝 $H_0$。

---

#### 7. 何时用哪一个？（实务建议）
- **优先 Welch**：当不确定方差齐性、样本量不平衡、或观察到两组方差差异较大时。  
- **可用 Student**：有充分理由认为两组方差相等（或经检验/领域知识支持），且样本量不小；此时统计功效更高。

---

#### 8. 常见易错点
- 把 **SE** 当作总体标准差；其实 SE 是**估计量的标准差**（抽样波动规模），非数据的离散度。  
- 方差不齐仍强行用合并方差：可能低估 SE，导致第一类错误膨胀。  
- “不拒绝 $H_0$” ≠ “$H_0$ 为真”。只是证据不足。

---

#### 9. 公式速记卡
- 合并方差：  
  $$
  S_p^2=\dfrac{(n_1-1)S_X^2+(n_2-1)S_Y^2}{n_1+n_2-2}.
  $$
- SE（齐性 / 不齐）：  
  $$
  \mathrm{SE}_{\text{pooled}}=S_p\sqrt{\tfrac{1}{n_1}+\tfrac{1}{n_2}},\quad
  \mathrm{SE}_{\text{Welch}}=\sqrt{\tfrac{S_X^2}{n_1}+\tfrac{S_Y^2}{n_2}}.
  $$
- t 统计量与 df：  
  $$
  t=\frac{(\bar X-\bar Y)-(\mu_1-\mu_2)}{\mathrm{SE}},\quad
  \text{df}=
  \begin{cases}
    n_1+n_2-2 & \text{(Student)}\\[4pt]
    \nu\ \text{(Welch)} & 
  \end{cases}
  $$
  $$
  \nu=\dfrac{\left(\tfrac{S_X^2}{n_1}+\tfrac{S_Y^2}{n_2}\right)^2}
  {\tfrac{(S_X^2/n_1)^2}{n_1-1}+\tfrac{(S_Y^2/n_2)^2}{n_2-1}}.
  $$

---

#### 10. 一句话小结
> 两独立样本均值比较的核心是：用恰当的 **SE** 标准化均值差形成 **t 统计量**。  
> **齐性**用合并方差（功效高、df 整数），**不齐**用 Welch（稳健、df 近似）。  
> 根据临界值或 p 值作出“拒绝 / 不拒绝 $H_0$”的判定。